# JWST Pipeline Validation Testing notebook: flat_field step with MIRI Imaging

Instruments Affected: NIRCam, NIRSpec, NIRISS, MIRI, FGS

## Introduction and summary of test being run

This notebook processes an image through calwebb_image2 (calwebb_detector1 is optional) and examines the output of the flat_field step. The steps are as follow:

1) Set up data path and directory and image file name.

2) Modify header information of input simulations (if needed).

3) Run input data through calwebb_detector1. (Can be done prior to running this notebook.)

4) Run output of calwebb_detector1 through the flat_field step in calwebb_image2.

5) Get flat field reference file. 

6) Compare the flat field reference file with the rate/cal image ratio and check that they are the same.

## Documentation

The pipeline documentation can be found here: https://jwst-pipeline.readthedocs.io/en/latest/

The pipeline code is available on GitHub: https://github.com/spacetelescope/jwst

## Data used 

The data used in this test is a simulated MIRI image created using MIRISim. The documentation for MIRISim can be found here: https://wiki.miricle.org/bin/view/Public/MIRISim_Public

Author: T. Temim

In [ ]:
# Create a temporary directory to hold notebook output, and change the working directory to that directory.
from tempfile import TemporaryDirectory
import os
data_dir = TemporaryDirectory()
os.chdir(data_dir.name)
print(data_dir)

### Set up import statements

In [ ]:
from astropy.io import fits, ascii
import pytest
import numpy as np

import jwst
from jwst.pipeline import Detector1Pipeline, Image2Pipeline
from jwst.datamodels import RampModel, ImageModel, dqflags
from jwst.pipeline import calwebb_image2

from ci_watson.artifactory_helpers import get_bigdata
import crds
import os

In [ ]:
# Specify CRDS locations and pmap
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
os.environ['CRDS_CONTEXT']='jwst_0619.pmap'
os.environ['TEST_BIGDATA']='https://bytesalad.stsci.edu/artifactory/'

### Print pipeline version number

In [ ]:
jwst.__version__ 

### Read in data from artifactory

In [ ]:
input_file = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'source_catalog',
                     'source_catalog_miri_test', 
                     'det_image_seq1_MIRIMAGE_F560Wexp1_rate.fits')

# Put in new simulation once we're sure MIRISim and the pipeline are using the same input flats.
#input_file = get_bigdata('jwst_validation_notebooks',
#                     'validation_data',
#                     'flat_field',
#                     'flat_field_miri_test', 
#                     'starfield_50star4ptdither_seq1_MIRIMAGE_F1130Wexp1_rate.fits')

### Read in input image as JWST data model

In [ ]:
im = ImageModel(input_file)

### Modify header information of input simulations (if needed)

In [ ]:
print(im.meta.wcsinfo.wcsaxes)

In [ ]:
im.meta.wcsinfo.wcsaxes=2

In [ ]:
print(im.meta.wcsinfo.wcsaxes)

In [ ]:
del im.meta.wcsinfo.cdelt3
del im.meta.wcsinfo.crpix3
del im.meta.wcsinfo.crval3
del im.meta.wcsinfo.ctype3
del im.meta.wcsinfo.cunit3
del im.meta.wcsinfo.pc3_1
del im.meta.wcsinfo.pc3_2

### Run input data through calwebb_detector1  (not done here)

In [ ]:
#det1 = calwebb_detector1.Detector1Pipeline()
#det1.save_results = True
#det1.run(im)

### Run output of calwebb_detector1 through calwebb_image2

In [ ]:
#input_file = input_file.replace('rateint.fits', 'rate.fits')

In [ ]:
im2 = calwebb_image2.Image2Pipeline()
#im2.background.skip = True
im2.assign_wcs.skip = True
im2.flat_field.skip = False
im2.photom.skip=True
im2.resample.skip = True
im2.save_results = True
im2.run(im)

In [ ]:
input_file = input_file.replace('rate.fits', 'cal.fits')

In [ ]:
im_cal = ImageModel(input_file)

### Calculate the rate/cal image ratio

In [ ]:
ratio_im=im.data/im_cal.data

### Get flat_field reference file

In [ ]:
flatreffile = im_cal.meta.ref_file.flat.name
print('Flat reference file', flatreffile)
# find location of file
basename = crds.core.config.pop_crds_uri(flatreffile)
path = crds.locate_file(basename, "jwst")
# open reference file
flat_im = ImageModel(path)

### Compare flat field reference file with the rate/cal image ratio and check that they are equal

In [ ]:
check_flat=ratio_im/flat_im.data

In [ ]:
print('Minimum and maximum values in the check_flat image are:', np.nanmin(check_flat), 'and', np.nanmax(check_flat))

### Both values above should equal to 1.0

### Check DQ flagging 
Any pixel flagged as NON_SCIENCE should also be flagged as DO_NOT_USE. Check if this is in place in both the input reference file and for the output science file of the calwebb_image2 pipeline. If there are no assert errors, the test below passes.

In [ ]:
# Check if the output cal file is flagged properly

# Test that all pixels flagged with NON_SCIENCE are also flagged as DO_NOT_USE
nonsciencearray = (im_cal.dq & dqflags.pixel['NON_SCIENCE'] > 0)
badarray = (im_cal.dq & dqflags.pixel['DO_NOT_USE'] > 0)
assert nonsciencearray.all() == badarray.all()

# Test if the input reference file had the flags all set the same way

nonsciencearray = (flat_im.dq & dqflags.pixel['NON_SCIENCE'] > 0)
badarray = (flat_im.dq & dqflags.pixel['DO_NOT_USE'] > 0)
assert nonsciencearray.all() == badarray.all()